In [ ]:
library(rtracklayer)
library(DESeq2)
library(dplyr)
library(ggplot2)
library(ggpubr)

# Mapping data

In [ ]:
gtf.mouse <- import("/projects/mludwig/DVC/data/bulk_rna-seq/raw/reference/gencode.vM23.annotation.gtf")
gtf.mouse <- as.data.frame(gtf.mouse)

gtf.rat <- import("/projects/mludwig/DVC/data/bulk_rna-seq/raw/reference/Rattus_norvegicus.Rnor_6.0.104.gtf")
gtf.rat <- as.data.frame(gtf.rat)

# Process bulk transcriptomics data from mouse

In [ ]:
# Read bulk data
main.dir <- "/projects/mludwig/DVC/data/bulk_rna-seq/raw/out/mouse/"
samples <- list.dirs(main.dir, recursive = F, full.names = F) 
suffix <- "ReadsPerGene.out.tab"
full.path <- paste0(main.dir, samples, "/", samples, suffix)

counts.list <- lapply(full.path, function(x) {read.csv(x, sep = "", skip = 4, header = F,
                                                       colClasses = c("character", "numeric", "NULL", "NULL"),
                                                       row.names = 1)})

counts.mouse <- do.call(cbind, counts.list)
names <- gsub("MGE01_|Grp.*", "", samples)
colnames(counts.mouse) <- names
counts.mouse <- counts.mouse[, paste0("mouse", 1:70)]

# Remove genes with zero variance
idx <- which(apply(counts.mouse, 1, var) != 0)
counts.mouse <- counts.mouse[idx, ] 

## Map to gene names
gene.names <- gtf.mouse$gene_name[match(rownames(counts.mouse), gtf.mouse$gene_id)]

# Keep gene id if unique mapping is not possible
keep.idx <- which(gene.names %in% unique(gene.names[duplicated(gene.names)]))
rownames(counts.mouse)[-keep.idx] <- gene.names[-keep.idx]

# Save
write.csv(counts.mouse, file = "/projects/mludwig/DVC/data/bulk_rna-seq/processed/counts_mouse.csv")

In [ ]:
# Load treatment groups
groups.mouse <- openxlsx::read.xlsx("/projects/mludwig/DVC/data/bulk_rna-seq/raw/groups_mouse.xlsx")
groups.mouse <- groups.mouse[, c(1, 3)]
colnames(groups.mouse) <- c("sample", "treatment")
groups.mouse$sample <- paste0("mouse", groups.mouse$sample)
groups.mouse$treatment <- gsub(1, "V-A", groups.mouse$treatment)
groups.mouse$treatment <- gsub(2, "A8-A", groups.mouse$treatment)
groups.mouse$treatment <- gsub(3, "A8-C", groups.mouse$treatment)
groups.mouse$treatment <- gsub(4, "WM-C", groups.mouse$treatment)
groups.mouse$treatment <- gsub(5, "L-A", groups.mouse$treatment)
groups.mouse$treatment <- as.factor(groups.mouse$treatment)

# Load QC data from different sequencing batches
main.dir <- "/projects/mludwig/DVC/data/bulk_rna-seq/raw/out/mouse/"
samples <- list.dirs(main.dir, recursive = F, full.names = F) 
suffix <- "Log.final.out"

mapped.read.path <- paste0(main.dir, samples, "/", samples, suffix)

QC.mouse1 <- data.frame(matrix(NA, nrow = 70, ncol = 2))
colnames(QC.mouse1) <- c("sample", "mapped.reads")
for (i in 1:length(samples)) {
  path <-  paste0(main.dir, samples[i], "/", samples[i], suffix)
  mapped.reads <- read.csv(path, header = F, colClasses = c(NA, "NULL"))[9,]
  mapped.reads <- as.numeric(gsub(".*Uniquely mapped reads % \\|\t|%$", "", mapped.reads))
  QC.mouse1$sample[i] <- samples[i]
  QC.mouse1$mapped.reads[i] <- mapped.reads
}
QC.mouse1$sample <- gsub("MGE01_|Grp.*", "", QC.mouse1$sample)

QC.mouse2 <- read.csv("/projects/mludwig/DVC/data/bulk_rna-seq/raw/90-612232719_statistics.csv")
QC.mouse2 <- QC.mouse2[grep("mouse", QC.mouse2$Sample.ID), ]
colnames(QC.mouse2) <- c("project", "sample", "barcode", "reads", "yield", "quality.score", "bases")
QC.mouse2 <- QC.mouse2[, c("sample", "reads", "yield", "quality.score", "bases")]
QC.mouse2$sample <- gsub("MGE01_|Grp.*", "", QC.mouse2$sample)
QC.mouse2$reads <- as.numeric(gsub("\\,", "", QC.mouse2$reads))
QC.mouse2$yield <- as.numeric(gsub("\\,", "", QC.mouse2$yield))

QC.mouse3 <- read.csv("/projects/mludwig/DVC/data/bulk_rna-seq/raw/90-612232719_rRNA.txt", sep = "\t")
QC.mouse3 <- QC.mouse3[grep("mouse", QC.mouse3$Sample.ID), ]
colnames(QC.mouse3) <- c("sample", "rRNA")
QC.mouse3$sample <- gsub("MGE01_|Grp.*", "", QC.mouse3$sample)
QC.mouse3$rRNA <- as.numeric(gsub("\\,", "\\.", QC.mouse3$rRNA))

QC.mouse <- merge(QC.mouse1, QC.mouse2, by = "sample")
QC.mouse <- merge(QC.mouse, QC.mouse3, by = "sample")
QC.mouse <- QC.mouse[match(paste0("mouse", 1:70), QC.mouse$sample),]
QC.mouse


# Merge
meta.mouse <- merge(x = groups.mouse, y = QC.mouse, by = "sample")
meta.mouse <- meta.mouse[match(groups.mouse$sample, meta.mouse$sample), ]

# Save
write.csv(meta.mouse, file = "/projects/mludwig/DVC/data/bulk_rna-seq/processed/meta_mouse.csv", quote = F, row.names = F)

# Process bulk transcriptomics data from rat

In [ ]:
# Read bulk data
main.dir <-  "/projects/mludwig/DVC/data/bulk_rna-seq/raw/out/rat/"
samples <- list.dirs(main.dir, recursive = F, full.names = F) 
suffix <- "ReadsPerGene.out.tab"
full.path <- paste0(main.dir, samples, "/", samples, suffix)


counts.list <- lapply(full.path, function(x) {read.csv(x, sep = "", skip = 4, header = F,
                                                       colClasses = c("character", "numeric", "NULL", "NULL"),
                                                       row.names = 1)})

counts.rat <- do.call(cbind, counts.list)
names <-  gsub("MGE01_|Grp.*", "", samples)
names <- gsub("Rat", "rat", names)
colnames(counts.rat) <- names 
counts.rat <- counts.rat[, paste0("rat", 1:70)]

# Remove genes with zero variance
idx <- which(apply(counts.rat, 1, var) != 0)
counts.rat <- counts.rat[idx, ] 

## Map to gene names
gene.names <- gtf.rat$gene_name[match(rownames(counts.rat), gtf.rat$gene_id)]

# Keep gene id if unique mapping is not possible
keep.idx <- which(gene.names %in% unique(gene.names[duplicated(gene.names)]))

rownames(counts.rat)[-keep.idx] <- gene.names[-keep.idx]

# Save
write.csv(counts.rat, file = "/projects/mludwig/DVC/data/bulk_rna-seq/processed/counts_rat.csv")

In [ ]:
# Load treatment groups
groups.rat <- openxlsx::read.xlsx("/projects/mludwig/DVC/data/bulk_rna-seq/raw/groups_rat.xlsx")
colnames(groups.rat) <- c("sample", "treatment")
groups.rat$sample <- paste0("rat", groups.rat$sample)
groups.rat$treatment <- gsub(1, "V-A", groups.rat$treatment)
groups.rat$treatment <- gsub(2, "A8-A", groups.rat$treatment)
groups.rat$treatment <- gsub(3, "A8-C", groups.rat$treatment)
groups.rat$treatment <- gsub(4, "WM-C", groups.rat$treatment)
groups.rat$treatment <- gsub(5, "L-A", groups.rat$treatment)


# Load QC data from different sequencing batches
main.dir <- "/projects/mludwig/DVC/data/bulk_rna-seq/raw/out/rat/"
samples <- list.dirs(main.dir, recursive = F, full.names = F) 
suffix <- "Log.final.out"

mapped.read.path <- paste0(main.dir, samples, "/", samples, suffix)

QC.rat1 <- data.frame(matrix(NA, nrow = 70, ncol = 2))
colnames(QC.rat1) <- c("sample", "mapped.reads")
for (i in 1:length(samples)) {
  path <-  paste0(main.dir, samples[i], "/", samples[i], suffix)
  mapped.reads <- read.csv(path, header = F, colClasses = c(NA, "NULL"))[9,]
  mapped.reads <- as.numeric(gsub(".*Uniquely mapped reads % \\|\t|%$", "", mapped.reads))
  QC.rat1$sample[i] <- samples[i]
  QC.rat1$mapped.reads[i] <- mapped.reads
}
QC.rat1$sample <- gsub("MGE01_|Grp.*", "", QC.rat1$sample)
QC.rat1$sample <- gsub("Rat", "rat", QC.rat1$sample)

QC.rat2 <- read.csv("/projects/mludwig/DVC/data/bulk_rna-seq/raw/90-612232719_statistics.csv")
QC.rat2 <- QC.rat2[grep("Rat", QC.rat2$Sample.ID), ]
colnames(QC.rat2) <- c("project", "sample", "barcode", "reads", "yield", "quality.score", "bases")
QC.rat2 <- QC.rat2[, c("sample", "reads", "yield", "quality.score", "bases")]
QC.rat2$sample <- gsub("MGE01_|Grp.*", "", QC.rat2$sample)
QC.rat2$sample <- gsub("Rat", "rat", QC.rat2$sample)
QC.rat2$reads <- as.numeric(gsub("\\,", "", QC.rat2$reads))
QC.rat2$yield <- as.numeric(gsub("\\,", "", QC.rat2$yield))

QC.rat3 <- read.csv("/projects/mludwig/DVC/data/bulk_rna-seq/raw/90-612232719_rRNA.txt", sep = "\t")
QC.rat3 <- QC.rat3[grep("Rat", QC.rat3$Sample.ID), ]
colnames(QC.rat3) <- c("sample", "rRNA")
QC.rat3$sample <- gsub("MGE01_|Grp.*", "", QC.rat3$sample)
QC.rat3$sample <- gsub("Rat", "rat", QC.rat3$sample)
QC.rat3$rRNA <- as.numeric(gsub("\\,", "\\.", QC.rat3$rRNA))

QC.rat <- merge(QC.rat1, QC.rat2, by = "sample")
QC.rat <- merge(QC.rat, QC.rat3, by = "sample")
QC.rat <- QC.rat[match(paste0("rat", 1:70), QC.rat$sample),]
QC.rat


# Merge
meta.rat <- merge(x = groups.rat, y = QC.rat, by = "sample")
meta.rat <- meta.rat[match(groups.rat$sample, meta.rat$sample), ]

# Save
write.csv(meta.rat, file = "/projects/mludwig/DVC/data/bulk_rna-seq/processed/meta_rat.csv", quote = F, row.names = F)

# Identify outliers in mouse transcriptomics data

In [ ]:
design <- data.frame(condition = meta.mouse$treatment, rRNA = meta.mouse$rRNA)
dds <- DESeqDataSetFromMatrix(counts.mouse, DataFrame(design), ~ condition + rRNA)
dds <- DESeq(dds)
vsd <- vst(dds, blind=F)

pca.data <- plotPCA(vsd, intgroup=c("condition"), returnData=TRUE, ntop = nrow(assay(vsd)))
percent.var <- round(100 * attr(pca.data, "percentVar"))
pca.data$condition <- factor(pca.data$condition)
pca.data$rRNA <- meta.mouse$rRNA

label <- data.frame(condition = unique(pca.data$condition))

label <- pca.data %>% 
  dplyr::group_by(condition) %>% 
  dplyr::summarize(x = median(PC1), y = median(PC2)) 

pca.plot <- ggplot(pca.data, aes(PC1, PC2, color = rRNA)) +
  geom_point(size = 2) +
  xlab(paste0("PC1 (", percent.var[1], "% variance)")) +
  ylab(paste0("PC2 (", percent.var[2], "% variance)")) + 
  theme_pubr() + theme(panel.border = element_blank(), 
                     panel.grid.major = element_blank(),
                     panel.grid.minor = element_blank(), 
                     axis.line = element_line(colour = "black"),
                     axis.text = element_text(hjust = 1, face = "bold"),
                     axis.title = element_text(size=10, face = "bold"),
                     legend.title = element_blank()) +   
  scale_x_continuous(breaks = seq(-500, 500, 10)) 
pca.plot

# Remove outliers with > 3 SD on PC1 and PC2
pca.data2 <- prcomp(t(assay(vsd)))
head(apply(pca.data2$x, 2, function(x) which( abs(x - mean(x)) > (3 * sd(x)))), n = 2)


# Mouse11 and mouse33 are identified as outliers and are removed (first iteration)
# Mouse25 and mouse27 are identified as outliers and are removed (second iteration)
samples.sub <- colnames(counts.mouse)[!(colnames(counts.mouse) %in% c("mouse11", "mouse25", "mouse27", "mouse33"))]
counts.sub <- counts.mouse[, samples.sub]
meta.sub <- meta.mouse[match(samples.sub, meta.mouse$sample), ]

design <- data.frame(condition = meta.sub$treatment, rRNA = meta.sub$rRNA)
dds <- DESeqDataSetFromMatrix(counts.sub, DataFrame(design), ~ condition + rRNA)
dds <- DESeq(dds)
vsd <- vst(dds, blind=F)

pca.data <- plotPCA(vsd, intgroup=c("condition"), returnData=TRUE, ntop = nrow(assay(vsd)))
percent.var <- round(100 * attr(pca.data, "percentVar"))
pca.data$condition <- factor(pca.data$condition)
pca.data$rRNA <- meta.sub$rRNA

label <- data.frame(condition = unique(pca.data$condition))

label <- pca.data %>% 
  dplyr::group_by(condition) %>% 
  dplyr::summarize(x = median(PC1), y = median(PC2)) 

pca.plot <- ggplot(pca.data, aes(PC1, PC2, color = rRNA)) +
  geom_point(size = 2) +
  xlab(paste0("PC1 (", percent.var[1], "% variance)")) +
  ylab(paste0("PC2 (", percent.var[2], "% variance)")) + 
  theme_pubr() + theme(panel.border = element_blank(), 
                     panel.grid.major = element_blank(),
                     panel.grid.minor = element_blank(), 
                     axis.line = element_line(colour = "black"),
                     axis.text = element_text(hjust = 1, face = "bold"),
                     axis.title = element_text(size=10, face = "bold"),
                     legend.title = element_blank()) +   
  scale_x_continuous(breaks = seq(-500, 500, 10)) 
pca.plot

# Remove outliers with > 3 SD on PC1 and PC2
pca.data2 <- prcomp(t(assay(vsd)))
head(apply(pca.data2$x, 2, function(x) which( abs(x - mean(x)) > (3 * sd(x)))), n = 2)

# Identify outliers in rat transcriptomics data

In [ ]:
design <- data.frame(condition = meta.rat$treatment, rRNA = meta.rat$rRNA)
dds <- DESeqDataSetFromMatrix(counts.rat, DataFrame(design), ~ condition + rRNA)
dds <- DESeq(dds)
vsd <- vst(dds, blind=F)

pca.data <- plotPCA(vsd, intgroup=c("condition"), returnData=TRUE, ntop = nrow(assay(vsd)))
percent.var <- round(100 * attr(pca.data, "percentVar"))
pca.data$condition <- factor(pca.data$condition)
pca.data$rRNA <- meta.rat$rRNA

label <- data.frame(condition = unique(pca.data$condition))

label <- pca.data %>% 
  dplyr::group_by(condition) %>% 
  dplyr::summarize(x = median(PC1), y = median(PC2)) 

pca.plot <- ggplot(pca.data, aes(PC1, PC2, color = rRNA)) +
  geom_point(size = 2) +
  xlab(paste0("PC1 (", percent.var[1], "% variance)")) +
  ylab(paste0("PC2 (", percent.var[2], "% variance)")) + 
  theme_pubr() + theme(panel.border = element_blank(), 
                     panel.grid.major = element_blank(),
                     panel.grid.minor = element_blank(), 
                     axis.line = element_line(colour = "black"),
                     axis.text = element_text(hjust = 1, face = "bold"),
                     axis.title = element_text(size=10, face = "bold"),
                     legend.title = element_blank()) +   
  scale_x_continuous(breaks = seq(-500, 500, 10)) 
pca.plot

# Remove outliers with > 3 SD on PC1 and PC2
pca.data2 <- prcomp(t(assay(vsd)))
head(apply(pca.data2$x, 2, function(x) which( abs(x - mean(x)) > (3 * sd(x)))), n = 2)


# Rat9 and rat65 are identified as outliers and are removed
samples.sub <- colnames(counts.rat)[!(colnames(counts.rat) %in% c("rat9", "rat65"))]
counts.sub <- counts.rat[, samples.sub]
meta.sub <- meta.rat[meta.rat$sample %in% samples.sub, ]

design <- data.frame(condition = meta.sub$treatment, rRNA = meta.sub$rRNA)
dds <- DESeqDataSetFromMatrix(counts.sub, DataFrame(design), ~ condition + rRNA)
dds <- DESeq(dds)
vsd <- vst(dds, blind=F)

pca.data <- plotPCA(vsd, intgroup=c("condition"), returnData=TRUE, ntop = nrow(assay(vsd)))
percent.var <- round(100 * attr(pca.data, "percentVar"))
pca.data$condition <- factor(pca.data$condition)
pca.data$rRNA <- meta.sub$rRNA

label <- data.frame(condition = unique(pca.data$condition))

label <- pca.data %>% 
  dplyr::group_by(condition) %>% 
  dplyr::summarize(x = median(PC1), y = median(PC2)) 

pca.plot <- ggplot(pca.data, aes(PC1, PC2, color = rRNA)) +
  geom_point(size = 2) +
  xlab(paste0("PC1 (", percent.var[1], "% variance)")) +
  ylab(paste0("PC2 (", percent.var[2], "% variance)")) + 
  theme_pubr() + theme(panel.border = element_blank(), 
                     panel.grid.major = element_blank(),
                     panel.grid.minor = element_blank(), 
                     axis.line = element_line(colour = "black"),
                     axis.text = element_text(hjust = 1, face = "bold"),
                     axis.title = element_text(size=10, face = "bold"),
                     legend.title = element_blank()) +   
  scale_x_continuous(breaks = seq(-500, 500, 10)) 
pca.plot

# No more outliers are detected
pca.data2 <- prcomp(t(assay(vsd)))
head(apply(pca.data2$x, 2, function(x) which( abs(x - mean(x)) > (3 * sd(x)))), n = 2)